# Chocolate Sales Forecasting — Modeling & Evaluation (2022–2023 → 2024)

This notebook builds and evaluates forecasting models using a time-aware approach.  
Main steps:
- Aggregating transactions into a monthly dataset per country
- Creating a proper time index (`ds`) and engineering lag features (e.g., `lag_1`, `lag_2`)
- Adding seasonality features (cyclical encoding with `month_sin` / `month_cos`)
- Training and comparing models (Naive baseline, Ridge, HistGradientBoosting)
- Evaluating performance on a future holdout set (train: 2022–2023, test: 2024) using RMSE and MAPE

**Output:** final trained models (revenue + boxes) and saved artifacts (features list, monthly dataset).

In [16]:
import pandas as pd

df = pd.read_csv("../data/processed_data.csv")
df.head()

,Country,Product,Date,Amount,Boxes Shipped,Amount_per_box,Month,Month_Name,Year
0,UK,Mint Chip Choco,2022-01-04,5320.0,180,29.555556,1,January,2022
1,India,85% Dark Bars,2022-08-01,7896.0,94,84.000000,8,August,2022
2,India,Peanut Butter Cubes,2022-07-07,4501.0,91,49.461538,7,July,2022
3,Australia,Peanut Butter Cubes,2022-04-27,12726.0,342,37.210526,4,April,2022
4,UK,Peanut Butter Cubes,2022-02-24,13685.0,184,74.375000,2,February,2022


In [17]:
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)

df.head()

,country,product,date,amount,boxes_shipped,amount_per_box,month,month_name,year
0,UK,Mint Chip Choco,2022-01-04,5320.0,180,29.555556,1,January,2022
1,India,85% Dark Bars,2022-08-01,7896.0,94,84.000000,8,August,2022
2,India,Peanut Butter Cubes,2022-07-07,4501.0,91,49.461538,7,July,2022
3,Australia,Peanut Butter Cubes,2022-04-27,12726.0,342,37.210526,4,April,2022
4,UK,Peanut Butter Cubes,2022-02-24,13685.0,184,74.375000,2,February,2022


In [18]:
monthly = (df.groupby(["year","month", "country"], as_index=False)
             .agg(total_revenue=("amount", "sum"),
                  total_boxes=("boxes_shipped", "sum")))

monthly.head(10)

,year,month,country,total_revenue,total_boxes
0,2022,1,Australia,187383.0,6449
1,2022,1,Canada,143997.0,4455
2,2022,1,India,143430.0,4354
3,2022,1,New Zealand,124488.0,5516
4,2022,1,UK,188531.0,4173
5,2022,1,USA,108276.0,2588
6,2022,2,Australia,126406.0,2635
7,2022,2,Canada,134456.0,3563
8,2022,2,India,83888.0,2486
9,2022,2,New Zealand,132293.0,3138


In [19]:
# date-stamp :

monthly["ds"] = pd.to_datetime(
        monthly["year"].astype(str) + "-" + monthly["month"].astype(str) + "-01"
    )

monthly = monthly.sort_values(["country", "ds"]).reset_index(drop=True)


# 1) Create lag features per country
lags = [1, 2]

for lag in lags:
    monthly[f"rev_lag_{lag}"] = monthly.groupby("country")["total_revenue"].shift(lag)
    monthly[f"box_lag_{lag}"] = monthly.groupby("country")["total_boxes"].shift(lag)

monthly.drop(columns=["year", "month"], errors="ignore", inplace=True)
monthly.head()

,country,total_revenue,total_boxes,ds,rev_lag_1,box_lag_1,rev_lag_2,box_lag_2
0,Australia,187383.0,6449,2022-01-01,NaN,NaN,NaN,NaN
1,Australia,126406.0,2635,2022-02-01,187383.0,6449.0,NaN,NaN
2,Australia,165431.0,4309,2022-03-01,126406.0,2635.0,187383.0,6449.0
3,Australia,120561.0,4470,2022-04-01,165431.0,4309.0,126406.0,2635.0
4,Australia,124264.0,4218,2022-05-01,120561.0,4470.0,165431.0,4309.0


In [20]:
lag_cols = [c for c in monthly.columns if c.startswith("rev_lag_") or c.startswith("box_lag_")]

print("Shape before:", monthly.shape)
monthly = monthly.dropna(subset=lag_cols)
print("Shape after :", monthly.shape)

Shape before: (144, 8)
Shape after : (132, 8)


In [21]:
monthly["month_num"] = monthly["ds"].dt.month

# Time split: train = 2022-2023, test = 2024
train_mask = monthly["ds"].dt.year.isin([2022, 2023])
test_mask  = monthly["ds"].dt.year == 2024

# Define features X and target y (for revenue)
feature_cols = ["country", "month_num", "rev_lag_1", "rev_lag_2", "box_lag_1", "box_lag_2"]

X_train = monthly.loc[train_mask, feature_cols]
y_train = monthly.loc[train_mask, "total_revenue"]

X_test  = monthly.loc[test_mask, feature_cols]
y_test  = monthly.loc[test_mask, "total_revenue"]

print("Train:", X_train.shape, "| Test:", X_test.shape)
print("Train ds:", monthly.loc[train_mask, "ds"].min(), "to", monthly.loc[train_mask, "ds"].max())
print("Test  ds:", monthly.loc[test_mask, "ds"].min(),  "to", monthly.loc[test_mask, "ds"].max())

Train: (84, 6) | Test: (48, 6)
Train ds: 2022-03-01 00:00:00 to 2023-08-01 00:00:00
Test  ds: 2024-01-01 00:00:00 to 2024-08-01 00:00:00


In [22]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import root_mean_squared_error
import numpy as np

# --- preprocessing ---
cat_cols = ["country"]
num_cols = [c for c in X_train.columns if c not in cat_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ],
    remainder="drop"
)
# --- model ---
ridge = Ridge(alpha=1.0, random_state=42)

pipe = Pipeline([
    ("prep", preprocess),
    ("model", ridge),
])

# --- fit / predict ---
pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)

# --- metrics ---
rmse = root_mean_squared_error(y_test, pred)

def mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

print("Ridge RMSE:", round(rmse, 2))
print("Ridge MAPE:", round(mape(y_test, pred), 2), "%")

Ridge RMSE: 34080.53
Ridge MAPE: 17.16 %


In [23]:
# Baseline: y_hat = last month's revenue
baseline_pred = X_test["rev_lag_1"].values

rmse_base = root_mean_squared_error(y_test, baseline_pred)

print("Naive (lag1) RMSE:", round(rmse_base, 2))
print("Naive (lag1) MAPE:", round(mape(y_test, baseline_pred), 2), "%")


Naive (lag1) RMSE: 47469.06
Naive (lag1) MAPE: 25.4 %


In [24]:
def add_month_cyclical(X):
    X = X.copy()
    m = X["month_num"].astype(int)
    X["month_sin"] = np.sin(2 * np.pi * m / 12)
    X["month_cos"] = np.cos(2 * np.pi * m / 12)
    return X

# add to train/test
X_train2 = add_month_cyclical(X_train)
X_test2  = add_month_cyclical(X_test)

display(X_train2.head())
display(X_test2.head())

,country,month_num,rev_lag_1,rev_lag_2,box_lag_1,box_lag_2,month_sin,month_cos
2,Australia,3,126406.0,187383.0,2635.0,6449.0,1.000000e+00,6.123234e-17
3,Australia,4,165431.0,126406.0,4309.0,2635.0,8.660254e-01,-5.000000e-01
4,Australia,5,120561.0,165431.0,4470.0,4309.0,5.000000e-01,-8.660254e-01
5,Australia,6,124264.0,120561.0,4218.0,4470.0,1.224647e-16,-1.000000e+00
6,Australia,7,145719.0,124264.0,3955.0,4218.0,-5.000000e-01,-8.660254e-01


,country,month_num,rev_lag_1,rev_lag_2,box_lag_1,box_lag_2,month_sin,month_cos
16,Australia,1,146225.51,140534.96,3253.0,3492.0,0.500000,8.660254e-01
17,Australia,2,211823.90,146225.51,6701.0,3253.0,0.866025,5.000000e-01
18,Australia,3,140600.05,211823.90,2725.0,6701.0,1.000000,6.123234e-17
19,Australia,4,188116.40,140600.05,4401.0,2725.0,0.866025,-5.000000e-01
20,Australia,5,135902.82,188116.40,4526.0,4401.0,0.500000,-8.660254e-01


In [25]:
# --- preprocessing ---
cat_cols = ["country"]
num_cols = [c for c in X_train2.columns if c not in cat_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ]
)

# --- model ---
ridge = Ridge(alpha=1.0)

pipe = Pipeline([
    ("prep", preprocess),
    ("model", ridge),
])

# --- fit / predict ---
pipe.fit(X_train2, y_train)
pred2 = pipe.predict(X_test2)

# --- metrics ---
rmse2 = root_mean_squared_error(y_test, pred2)

def mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = (y_true != 0)
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

print("Ridge (cyclical) RMSE:", round(rmse2, 2))
print("Ridge (cyclical) MAPE:", round(mape(y_test, pred2), 2), "%")

Ridge (cyclical) RMSE: 32026.77
Ridge (cyclical) MAPE: 17.05 %


In [26]:
# target = total_revenue

from sklearn.ensemble import HistGradientBoostingRegressor

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ]
)

hgb = HistGradientBoostingRegressor(random_state=42)

pipe_hgb = Pipeline([
    ("prep", preprocess),
    ("model", hgb),
])

pipe_hgb.fit(X_train2, y_train)
pred_hgb = pipe_hgb.predict(X_test2)

rmse_hgb = root_mean_squared_error(y_test, pred_hgb)

print("HGB RMSE:", round(rmse_hgb, 2))
print("HGB MAPE:", round(mape(y_test, pred_hgb), 2), "%")

HGB RMSE: 27961.06
HGB MAPE: 14.38 %


In [27]:
# target = total_boxes

y_train_boxes = monthly.loc[train_mask, "total_boxes"]
y_test_boxes  = monthly.loc[test_mask, "total_boxes"]

# Fit same pipeline but with boxes target
pipe_hgb.fit(X_train2, y_train_boxes)
pred_boxes = pipe_hgb.predict(X_test2)

rmse_boxes = root_mean_squared_error(y_test_boxes, pred_boxes)

print("HGB (Boxes) RMSE:", round(rmse_boxes, 2))
print("HGB (Boxes) MAPE:", round(mape(y_test_boxes, pred_boxes), 2), "%")

HGB (Boxes) RMSE: 742.54
HGB (Boxes) MAPE: 14.89 %


In [28]:
# Fit + export two separate models
from sklearn.base import clone
import joblib

# Fit
pipe_hgb_rev = clone(pipe_hgb).fit(X_train2, y_train)          # revenue target
pipe_hgb_box = clone(pipe_hgb).fit(X_train2, y_train_boxes)    # boxes target

# Export
joblib.dump(pipe_hgb_rev, "../models/hgb_revenue.joblib")
joblib.dump(pipe_hgb_box, "../models/hgb_boxes.joblib")

['../models/hgb_boxes.joblib']

In [29]:
monthly.to_csv("../data/monthly_data.csv", index=False)

In [30]:
import json

FEATURES = list(X_train2.columns)

with open("../models/features.json", "w") as f:
    json.dump(FEATURES, f, indent=2)